## Условие задачи:
Начальная популяция:кол-во особей всегда = 200.  
1.1. Случайная генерация  
Отбор особей:  
2.2. Выбрать только 20% самых приспособленных  
Скрещивание:  
3.2. Однородный (каждый бит от случайно выбранного родителя)  
Мутация:  
4.1. Инвертирование всех битов у 1 особи  
Формирование новой популяции (кол-во особей: константа)  
5.3 замена своих родителей  
Оценка результата  
Наступила сходимость (функция приспособленности лучшей особи в популяциях отличается не более, чем на 10%) или прошло 500 поколений.

In [46]:
from numpy import genfromtxt
import random
from random import choice, randint

In [47]:
def get_data():
    with open('13.txt') as f:
        settings = f.readline().split(' ')
    max_weight = int(settings[0])
    max_volume = int(settings[1])
    data = genfromtxt('13.txt', delimiter=' ', dtype=(int, float, int), skip_header=1)
    return {
        'max_weight': int(max_weight),
        'max_volume': float(max_volume),
        'items': data.tolist()
    }
data = get_data()
#предметы
items = sorted(data['items'], key=lambda x: x[2], reverse=True)
#данные о рюкзаке
max_w = data['max_weight']
max_v = data['max_volume']
print(max_w, max_v)
#количество грузов
n = 200
generations = 500
#print(items)

13000 12.0


Случайная генерация

In [48]:
def generate(item):
    individ = ""
    wei = val = vol = 0
    for i in range(len(item)):
        ind = random.randint(0, 1)
        if (ind):
            wei += item[i][0]
            vol += item[i][1]
            val += item[i][2]
            
        individ += str(ind)
    return {"gen": individ, "wei": wei, "vol": vol, "val": val}

population = []
for i in range(n):
    population.append(generate(items))
print(len(population))

200


Выбор для скрещивания (20% приспособленных особей)

In [49]:
#проверка подходит ли нам особь или нет (его приспособленность)
def validate(ind):
    weight = ind['wei']
    volume = ind['vol']
    if (weight >= max_w or volume >= max_v):
        return 0
    else:
        return 1


def selection(population, s=20):
    #считаем 20% от наших особей и сортируем по ценности 
    percent = int(len(population) * s / 100) 
    population = sorted(population, key = lambda i: i['val'],reverse=True)
    
    #выбранные, не выбранные особи и количество выбранных
    select = []
    not_select = []
    count = 0
                      
    for i in population:
        #если не подходит, то добавляем его в невыбранные особи
        if (validate(i) == 0):
            not_select.append(i)
        #если наше количество выбранных особей не привысило 20%
        elif(count < percent):
            select.append(i)
            count += 1
        else:
            not_select.append(i)            
    return  select,not_select

#sel, not_sel  = selection(population)
#print(sel)

Мутация: инвертирование битов

In [50]:
def mutate(ind):
    newChromo = ''
    for i in ind:
        if i=='0':
            gen = '1'
        else:
            gen = '0'
        newChromo += gen
    return {'gen': newChromo, 'wei': 0,  'vol': 0, 'val':0}

Скрещивание (кроссинговер) между выбранными особями. Каждая особь скрещивается 1 раз за 1 поколение, 1 пара дает 2 потомка :
однородный (каждый бит от случайно выбранного родителя)

In [51]:
def crossingover(parent1, parent2):
    parents = [parent1, parent2]
    #print(parents)
    childs = []
    for c in range(2):
        #маска, от какого родителя берем бит
        mask = []
        for i in range(len(parent1)):
            mask.append(random.randrange(2))
        child = ''
        j = 0
        for i in mask:
            child += parents[i][j]
            j += 1
        childs.append(child)
    return childs


def getOffspring(indToCross):
    offspring =[]
    while indToCross!=[]:
        ind1 = choice(indToCross)
        indToCross.remove(ind1)
        ind2 = choice(indToCross)
        indToCross.remove(ind2)
        childs= crossingover(ind1['gen'], ind2['gen'])
        offspring.append({'gen': childs[0], 'wei': 0,  'vol': 0, 'val':0})
        offspring.append({'gen': childs[1], 'wei': 0,  'vol': 0, 'val':0})
    mutInd = choice(offspring)
    offspring.remove(mutInd)
    offspring.append(mutate(mutInd['gen']))
    return offspring
#дети
#offspring = getOffspring(sel)
#print(len(offspring))
#print(offspring)

In [52]:
#подсчет функции приспособленности
def calcFitness(ind):
    summ_val = summ_vol = summ_wei = 0
    for i in range(len(items)):
        summ_wei += float(ind[i])*items[i][0]
        summ_vol += float(ind[i])*items[i][1]
        summ_val += float(ind[i])*items[i][2]
        if summ_vol<=max_v and summ_wei<=max_w:
            price = summ_val
        else:
            price=0
    
    return {'gen': ind, 'wei': summ_wei, 'vol': summ_vol, 'val': price}

#замена родителей детьми(к невыбранным особям добавляются дети)
#for i in offspring:
    #not_sel.append(calcFitness(i["gen"]))
#print(not_sel)

In [53]:
for i in range(generations):
    #print("Итерация: " + str(i))
    
    #выбор 20%
    sel, not_sel  = selection(population)
    #получение детей
    offspring = getOffspring(sel)
    #добавление детей к невыбранным особям (замена своих рдителей)
    for i in offspring:
        not_sel.append(calcFitness(i["gen"]))
    
sel, not_sel = selection(population)
population = sorted(sel, key = lambda i: i['val'],reverse=True)
print(population[0]['val'])
print(population[0]['vol'])
print(population[0]['wei'])    
    

4075
11.5
11263
